<a href="https://colab.research.google.com/github/Glasiermedic/DataPrep/blob/master/Dataset_preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing Needed Libraries

In [0]:
import numpy as np
import os

import pandas as pd
import matplotlib.pyplot as plt
import statistics
%matplotlib inline

import seaborn as sns
import re

from sklearn.model_selection import train_test_split
from sklearn.feature_selection import VarianceThreshold

Importing dataset: Student Performance

In [0]:
std_per_raw = pd.read_csv('https://raw.githubusercontent.com/Glasiermedic/DataPrep/master/StudentsPerformance.csv', encoding='latin1')

Examining  dataset for feature selection starts with finding out what kinds of data was collected, if there are any empty/null values entered and looking for a variable that readily answers the question.    Our question today is "Can we used standardized testing to predict the education level of parents?"

In [3]:
std_per_raw.describe()

,math score,reading score,writing score
count,1000.00000,1000.000000,1000.000000
mean,66.08900,69.169000,68.054000
std,15.16308,14.600192,15.195657
min,0.00000,17.000000,10.000000
25%,57.00000,59.000000,57.750000
50%,66.00000,70.000000,69.000000
75%,77.00000,79.000000,79.000000
max,100.00000,100.000000,100.000000


In [4]:
#provides a list of variables and their type

std_per_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
gender                         1000 non-null object
race/ethnicity                 1000 non-null object
parental level of education    1000 non-null object
lunch                          1000 non-null object
test preparation course        1000 non-null object
math score                     1000 non-null int64
reading score                  1000 non-null int64
writing score                  1000 non-null int64
dtypes: int64(3), object(5)
memory usage: 62.6+ KB


In [5]:
#provides a lists of each variable and if the variable has any na/null/empty entries
std_per_raw.isna().any()

gender                         False
race/ethnicity                 False
parental level of education    False
lunch                          False
test preparation course        False
math score                     False
reading score                  False
writing score                  False
dtype: bool

In [6]:
#provides a look at the first ten entries
std_per_raw.head(10)

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75
5,female,group B,associate's degree,standard,none,71,83,78
6,female,group B,some college,standard,completed,88,95,92
7,male,group B,some college,free/reduced,none,40,43,39
8,male,group D,high school,free/reduced,completed,64,64,67
9,female,group B,high school,free/reduced,none,38,60,50


In [7]:
#provide the standard deviation of each of the numerical variables
std_per_raw.std()

math score       15.163080
reading score    14.600192
writing score    15.195657
dtype: float64

In [8]:
#provide the variance for numerical variables
std_per_raw.var()

math score       229.918998
reading score    213.165605
writing score    230.907992
dtype: float64

In [9]:
#Examining the potential outcomes for parental education
std_per_raw['parental level of education'].unique()


array(["bachelor's degree", 'some college', "master's degree",
       "associate's degree", 'high school', 'some high school'],
      dtype=object)

Based on the data set we can use the "parental level of education" as our outcome.  We can use "math score", "reading score" and "writing score" along with "race/ethnicity" as our features.  We will create a "combined_score", "language score", "prepared_math","prepared_reading","prepared_writing, "prepared_language", prepared_combined" 

In [0]:
#change parental level of education to a category rather than an object variable type and test for various feature qualities
std_per_raw["parental level of education"] = std_per_raw["parental level of education"].astype('category')
#test= (std_per_raw['test preparation course']=='completed').astype('int')
#print(test.head(10))

In [0]:
# creating the combined score for all tests and then a prepared score variable for each type of test 
# for each type of test
std_per_raw['combined_score']=std_per_raw['math score']\
+ std_per_raw['reading score'] + std_per_raw['writing score']
std_per_raw['prepared_math']= std_per_raw['math score']/100 + \
(std_per_raw['test preparation course']=='completed').astype('int')
std_per_raw['prepared_read'] = std_per_raw['reading score']/100 + \
(std_per_raw['test preparation course']=='completed').astype('int')
std_per_raw['prepared_writing'] = std_per_raw['writing score']/100 + \
(std_per_raw['test preparation course']=='completed').astype('int')

In [0]:
#creating the combined score for writing and reading and labeling it language score
std_per_raw['language score'] = std_per_raw['reading score']\
+ std_per_raw['writing score']

In [0]:
#creating the average of math writing and reading scores
std_per_raw['averaged_scores'] = (std_per_raw['math score'] \
+ std_per_raw['reading score'] + std_per_raw['writing score']) / 3
#std_per_raw['averaged_scores'].head(10)

In [0]:
#creating the average language scores
std_per_raw['averaged_language'] = (std_per_raw['reading score']\
+ std_per_raw['writing score']) / 2
#std_per_raw['averaged_language'].head(10)

In [0]:
#std_per_raw.describe()

#### creating the "relative_reading" and "relative_writing" variables by fitting them to the lowest and highest scores recorded/possible.  
#### original scores x1 and x2 should become y1 and y2.  x1=max and x2=min  
#### y1=new max and y2 =new min  

###formula for linear scaling :
####y1 + ((y2-y1)/(x2-x1))(X-x1)


In [0]:
# creating a function to transform a variable in a database so it fits in a range of values
def lin_scl_min_max(db,variable,new_min,new_max):
  db['scaled {}'.format(variable)] = new_max + ((new_min-new_max)/ \
  (db['{}'.format(variable)].min()-db['{}'.format(variable)].max()))* \
  (db['{}'.format(variable)]-db['{}'.format(variable)].max())
  
 

In [0]:
#creating the scaled variables for reading and writing
lin_scl_min_max(std_per_raw,'reading score',0,100)
lin_scl_min_max(std_per_raw,'writing score',0,100)

In [0]:
#creating the percentile ranking for math reading and writing
std_per_raw['Percentile_math']=std_per_raw['math score'].rank(pct=True)
std_per_raw['Percentile_reading']=std_per_raw['reading score'].rank(pct=True)
std_per_raw['Percentile_writing']=std_per_raw['writing score'].rank(pct=True)

In [0]:
#std_per_raw.head(10)

### Now we are going to select the features that will give us the best results in our machine learning model. Since we are interested in correlations between features we need to remove all variables that are not numerical. Then we need to create a training and testing set of data from the master data set. 


In [0]:
num_colums = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64', 'category']  
numerical_columns = list(std_per_raw.select_dtypes(include=num_colums).columns)  
std_per_num = std_per_raw[numerical_columns] 

features = std_per_num.columns

In [21]:
features
#std_per_num.head()

Index(['parental level of education', 'math score', 'reading score',
       'writing score', 'combined_score', 'prepared_math', 'prepared_read',
       'prepared_writing', 'language score', 'averaged_scores',
       'averaged_language', 'scaled reading score', 'scaled writing score',
       'Percentile_math', 'Percentile_reading', 'Percentile_writing'],
      dtype='object')

In [0]:
X = std_per_num.drop(labels=['parental level of education'],axis =1)
y = std_per_num['parental level of education']

Creating the training and test data for our feature filtering method, with training taking on 

In [0]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=1)

#### Now we can look at all the features that are not constant  and are more useful for our purposes. 

In [24]:
constant_filter = VarianceThreshold(threshold=0)

constant_filter.fit(X_train)  

print("Number of features that are not constant: {}".format(len(X_train.columns[constant_filter.get_support()])))

Number of features that are not constant: 15


#### Now we will look at what features are constant and less useful for us to use for our algorhythms. 

In [25]:
constant_columns = [column for column in X_train.columns  
                    if column not in X_train.columns[constant_filter.get_support()]]

print("Number of features that are constant: {}".format(len(constant_columns)))

Number of features that are constant: 0


#### For the sake of defining a process of utilizing a filter method I am including the step to transform the dataset using our contant filter.  

In [26]:
#X_train = constant_filter.transform(X_train)  
#X_test = constant_filter.transform(X_test)

print("the dimensions of the training and test data sets respectively: {}{}".format(X_train.shape, X_test.shape))

the dimensions of the training and test data sets respectively: (800, 15)(200, 15)


####Now we will determine how many features are correlated and decide if we can remove any of them.  First we create the correlation matrix and then we will loop through the matrix adding columns with a correlation above .98 to our correlated_features set.  

In [27]:
correlated_features = set()  
correlation_matrix = std_per_raw.corr()

for i in range(len(correlation_matrix .columns)):  
    for j in range(i):
        if abs(correlation_matrix.iloc[i, j]) > 0.972 and i!= j:
            colname = correlation_matrix.columns[i]
            correlated_features.add(colname)
#print(correlation_matrix)
print(correlated_features) 
print(len(correlated_features))

{'Percentile_reading', 'prepared_writing', 'Percentile_math', 'language score', 'scaled writing score', 'prepared_read', 'Percentile_writing', 'averaged_scores', 'scaled reading score', 'averaged_language'}
10


#### After examining the results of the correlation filter we can see that ten of our features are highly correlated to at least one other feature and could be removed to help our model work more effectively.   This leaves just five features for us to use for our model. 

In [28]:
X_train.drop(labels=correlated_features, axis=1, inplace=True)  
X_test.drop(labels=correlated_features, axis=1, inplace=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


###Our prepared training and test set
####We have filtered our dataset down to five features that are the least correlated and have enough variance to produce good results.  

In [29]:
X_train.head(10)

,math score,reading score,writing score,combined_score,prepared_math
382,79,81,71,231,0.79
994,63,63,62,188,0.63
982,79,85,86,250,1.79
47,66,71,76,213,0.66
521,91,86,84,261,0.91
636,67,80,81,228,1.67
19,54,58,61,173,0.54
366,69,58,53,180,1.69
340,58,61,52,171,0.58
952,74,74,72,220,0.74
